In [1]:
import os
from pathling import PathlingContext

#MIMIC_FHIR_PATH="../_data/mimic4-demo-ptl"
MIMIC_FHIR_PATH = "/Users/szu004/datasets/work/mimic-iv/mimic4-ptl"

In [2]:
os.environ['SPARK_CONF_DIR'] = os.path.abspath('../env/spark-conf')
pc = PathlingContext.create()
spark = pc.spark
print('spark.sql.ansi.enabled', spark.conf.get("spark.sql.ansi.enabled"))
print('spark.sql.timestampType', spark.conf.get("spark.sql.timestampType"))

:: loading settings :: url = jar:file:/Users/szu004/miniconda3/envs/sof_mimic/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/szu004/.ivy2/cache
The jars for the packages stored in: /Users/szu004/.ivy2/jars
au.csiro.pathling#library-runtime added as a dependency
io.delta#delta-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1ea3b978-970f-4c43-a065-3bc3149886bb;1.0
	confs: [default]
	found au.csiro.pathling#library-runtime;7.1.0-SNAPSHOT in local-m2-cache
	found io.delta#delta-spark_2.12;3.2.0 in local-m2-cache
	found io.delta#delta-storage;3.2.0 in local-m2-cache
	found org.antlr#antlr4-runtime;4.9.3 in local-m2-cache
	found org.apache.hadoop#hadoop-aws;3.3.4 in local-m2-cache
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in local-m2-cache
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in local-m2-cache
:: resolution report :: resolve 134ms :: artifacts dl 6ms
	:: modules in use:
	au.csiro.pathling#library-runtime;7.1.0-SNAPSHOT from local-m2-cache in [default]
	co

2024-09-09 15:39:55 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-09-09 15:39:56 WARN  Utils:72 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2024-09-09 15:39:57 INFO  Version:21 - HV000001: Hibernate Validator null
2024-09-09 15:39:57 DEBUG AbstractConfigurationImpl:176 - Setting custom MessageInterpolator of type au.csiro.pathling.shaded.org.hibernate.validator.messageinterpolation.ParameterMessageInterpolator
2024-09-09 15:39:57 DEBUG ValidationXmlParser:122 - Trying to load META-INF/validation.xml for XML based Validator configuration.
2024-09-09 15:39:57 DEBUG ResourceLoaderHelper:61 - Trying to load META-INF/validation.xml via TCCL
2024-09-09 15:39:57 DEBUG ResourceLoaderHelper:67 - Trying to load META-INF/validation.xml via Hibernate Validator's class loader
2024-09-09 15:39:57 DEBUG ValidationXmlParser:129 - No META-INF/validation.xml found. Using annotation based c

In [3]:
data = pc.read.parquet(MIMIC_FHIR_PATH)
data.read('Patient').show()

2024-09-09 15:40:02 WARN  SparkStringUtils:72 - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+-------------+--------+----+--------------------+------+--------------------+-------+------+----------+---------------+----------------+-------+--------------------+--------------------+--------------------+-----+-------+--------------------+-------------------+--------------------+----+----------+--------------------+
|                  id|        id_versioned|                meta|implicitRules|language|text|          identifier|active|                name|telecom|gender| birthDate|deceasedBoolean|deceasedDateTime|address|       maritalStatus|multipleBirthBoolean|multipleBirthInteger|photo|contact|       communication|generalPractitioner|managingOrganization|link|      _fid|          _extension|
+--------------------+--------------------+--------------------+-------------+--------+----+--------------------+------+--------------------+-------+------+----------+---------------+----------------+-------+--------------------+-

In [24]:
import glob
import json
import re

from typing import NamedTuple

class ViewCtx(NamedTuple):
    spark: 'SparkSession'
    ds: 'DataSource'


VIEW_CTX = ViewCtx(spark, data)

class SOFView:
    def __init__(self, view_def, name=None):
        self._view_def = view_def
        self._name = name or view_def['name']

    def run(self, view_ctx):
        print(f"Creating SOF view {self._name}")
        view = view_ctx.ds.view(json=json.dumps(self._view_def))
        view.cache().createOrReplaceTempView(self._name)

    def select(self, view_ctx):
        return view_ctx.spark.sql(f"SELECT * FROM {self._name}")

    @classmethod
    def from_file(cls, file_path):
        with open(file_path) as f:
            view_def = json.load(f)
        return cls(view_def)

for view_file in glob.glob('../src/sof/*.json'):
    sof_view = SOFView.from_file(view_file)
    sof_view.run(VIEW_CTX)


class SQLView:
    def __init__(self, sql, name, depends_on=None):
        self._sql = sql
        self._name = name

    def run(self, view_ctx):
        print(f"Creating SQL view {self._name}")
        view_ctx.spark.sql(self._sql)

    def select(self, view_ctx):
        return view_ctx.spark.sql(f"SELECT * FROM {self._name}")


    def cache(self, view_ctx):
        return view_ctx.spark.sql(f"CACHE LAZY TABLE {self._name};")


    def to_csv(self, view_ctx, file_path):
        self.select(view_ctx).toPandas().to_csv(file_path, index = False)

    @classmethod
    def from_file(cls, file_path):
        with open(file_path) as f:
            sql = f.read()
        match = re.search(r'CREATE OR REPLACE \w* VIEW (\w+) AS', sql)
        if not match:
            raise ValueError(f"Cannot find view name in {file_path}")  
        return cls(sql, match.group(1))

for view_file in sorted(glob.glob('../src/sql/*.sql')):
    sof_view = SQLView.from_file(view_file)
    sof_view.run(VIEW_CTX)

SQLView.from_file('../src/sql/md_icustay_detail.sql').cache(VIEW_CTX)
SQLView.from_file('../src/sql/md_oxygen_delivery.sql').cache(VIEW_CTX)

#SQLView.from_file('../src/sql/md_icustay_detail.sql').select(VIEW_CTX).show()
#SQLView.from_file('../src/sql/dv_o2_delivery_device.sql').select(VIEW_CTX).show()


SQLView.from_file('../src/sql/xcoh_subject.sql').to_csv(VIEW_CTX, '../data/fhir_mimic-2.2/subject.csv')
SQLView.from_file('../src/sql/xcoh_x_reading_o2_flow.sql').to_csv(VIEW_CTX, '../data/fhir_mimic-2.2/reading_o2_flow.csv')

Creating SOF view rv_patient
2024-09-09 17:00:15 WARN  CacheManager:72 - Asked to cache already cached data.
Creating SOF view rv_o2_delivery_device
2024-09-09 17:00:15 WARN  CacheManager:72 - Asked to cache already cached data.
Creating SOF view rv_obs_o2_flow
2024-09-09 17:00:15 WARN  CacheManager:72 - Asked to cache already cached data.
Creating SOF view rv_icu_encounter
2024-09-09 17:00:15 WARN  CacheManager:72 - Asked to cache already cached data.
Creating SQL view dv_icu_encounter
Creating SQL view dv_o2_delivery_device
Creating SQL view dv_obs_o2_flow
Creating SQL view dv_patient
Creating SQL view md_icustay_detail
Creating SQL view md_oxygen_delivery
Creating SQL view st_subject
Creating SQL view stx_o2_flow
Creating SQL view coh_subject
Creating SQL view coh_reading_o2_flow


ERROR:root:KeyboardInterrupt while sending command.70][Stage 2217:>(0 + 0) / 11]
Traceback (most recent call last):
  File "/Users/szu004/miniconda3/envs/sof_mimic/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/szu004/miniconda3/envs/sof_mimic/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/szu004/miniconda3/envs/sof_mimic/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

## icustay_detail

### Patient

In [ ]:
rv_patient = data.view('Patient',
                       select=[
                           {
                               "column": [
                                   {
                                       "name": "patient_id",
                                       "path": "getResourceKey()",
                                       "collection": False
                                   },
                                   {
                                       "name": "gender",
                                       "path": "gender",
                                       "collection": False
                                   },
                                   {
                                       "name": "race_system",
                                       "path": "extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-race').extension('ombCategory').value.ofType(Coding).system",
                                       "collection": False
                                   },
                                   {
                                       "name": "race_code",
                                       "path": "extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-race').extension('ombCategory').value.ofType(Coding).code",
                                       "collection": False
                                   },
                                   {
                                       "name": "ethnicity_system",
                                       "path": "extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity').extension('ombCategory').value.ofType(Coding).system",
                                       "collection": False
                                   },
                                   {
                                       "name": "ethnicity_code",
                                       "path": "extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-race').extension('ombCategory').value.ofType(Coding).code",
                                       "collection": False
                                   },
                               ]
                           }
                       ]
                       )

rv_patient.createOrReplaceTempView('rv_patient')
rv_patient.toPandas()

In [ ]:
spark.sql("""
CREATE OR REPLACE TEMP VIEW dv_patient AS
SELECT patient_id AS subject_id,
    CASE 
        WHEN gender = 'male' THEN 'M'
        WHEN gender = 'female' THEN 'F'
    END AS gender,
    CASE 
        WHEN race_system = 'urn:oid:2.16.840.1.113883.6.238' THEN
            CASE 
                WHEN race_code = '2106-3' THEN 
                    CASE
                        WHEN ethnicity_system = 'urn:oid:2.16.840.1.113883.6.238' AND ethnicity_code = '2135-2' THEN 'HISPANIC' 
                        ELSE 'WHITE'
                    END
                WHEN race_code = '2054-5' THEN 'BLACK'
                WHEN race_code = '2028-9' THEN 'ASIAN'
            END
    END AS race
FROM rv_patient;    
""")
dv_patient = spark.sql("SELECT * FROM dv_patient")
dv_patient.toPandas()

### Encounter(ICU) 


In [ ]:
rv_icu_encounter = data.view('Encounter',
                             select=[
                                 {
                                     "column": [
                                         {
                                             "name": "encounter_id",
                                             "path": "getResourceKey()",
                                             "type": "string",
                                             "collection": False
                                         },
                                         {
                                             "name": "patient_id",
                                             "path": "subject.getReferenceKey()",
                                             "type": "string",
                                             "collection": False
                                         },
                                         {
                                             "name": "admit_date",
                                             "path": "period.start",
                                             "type": "dateTime",
                                             "collection": False
                                         },
                                         {
                                             "name": "disch_date",
                                             "path": "period.end",
                                             "type": "dateTime",
                                             "collection": False
                                         }

                                     ]
                                 }
                             ],
                             where=[
                                 {
                                     "path": "class.system = 'http://terminology.hl7.org/CodeSystem/v3-ActCode'"
                                 },
                                 {
                                     "path": "class.code = 'ACUTE'"
                                 },

                             ]
                             )

rv_icu_encounter.createOrReplaceTempView('rv_icu_encounter')
rv_icu_encounter.toPandas()

In [ ]:
dv_icu_encounter = spark.sql("""
WITH tv AS (
    SELECT 
        patient_id AS subject_id,
        encounter_id AS stay_id,
        CAST(admit_date AS TIMESTAMP) icu_intime, 
        CAST(disch_date  AS TIMESTAMP) icu_outtime 
    FROM rv_icu_encounter)
SELECT *, 
    row_number() over (PARTITION BY subject_id, stay_id ORDER BY icu_intime ASC) AS icustay_seq 
FROM tv
""")
dv_icu_encounter.createOrReplaceTempView('dv_icu_encounter')
dv_icu_encounter.toPandas()

### ICU Stay Detail

In [ ]:
icustay_detail = spark.sql("""
SELECT 
    enc.subject_id, enc.stay_id, pat.gender, pat. race, enc.icu_intime, enc.icu_outtime, enc.icustay_seq 
FROM dv_icu_encounter enc
LEFT OUTER JOIN dv_patient pat ON enc.subject_id = pat.subject_id
""")
icustay_detail.toPandas()
icustay_detail.write.parquet('../_data/mimic_derrived/icustay_detail.parquet', mode='overwrite')

## Oxygen Delivery Device

In [ ]:
CS_MIMIC_CHARTEVENTS_D_ITEMS = 'http://mimic.mit.edu/fhir/mimic/CodeSystem/mimic-chartevents-d-items'
CS_MIMIC_D_LABITEMS = 'http://mimic.mit.edu/fhir/mimic/CodeSystem/mimic-d-labitems'


def obs_str_for_coding(system, code, view_name=None):
    obs_view = data.view('Observation',
                         select=[
                             {
                                 "column": [
                                     {
                                         "name": "id",
                                         "path": "id",
                                         "type": "id",
                                         "collection": False
                                     },
                                     {
                                         "name": "patient_id",
                                         "path": "subject.getReferenceKey()",
                                         "type": "string",
                                         "collection": False
                                     },
                                     {
                                         "name": "encounter_id",
                                         "path": "encounter.getReferenceKey()",
                                         "type": "string",
                                         "collection": False
                                     },
                                     {
                                         "name": "charttime",
                                         "path": "effectiveDateTime",
                                         "type": "dateTime",
                                         "collection": False
                                     },
                                     {
                                         "name": "storetime",
                                         "path": "issued",
                                         "type": "instant",
                                         "collection": False
                                     },
                                     {
                                         "name": "value",
                                         "path": "valueString",
                                         "type": "string",
                                         "collection": False
                                     }
                                 ]
                             }
                         ],
                         where=[
                             {
                                 "path": f"code.coding.system = '{system}'"
                             },
                             {
                                 "path": f"code.coding.code = '{code}'"
                             },

                         ]
                         )
    if view_name is not None:
        obs_view.createOrReplaceTempView(view_name)
    return obs_view


rv_oxygen_delivery_device = obs_str_for_coding(CS_MIMIC_CHARTEVENTS_D_ITEMS, '226732', 'rv_oxygen_delivery_device')
rv_oxygen_delivery_device.toPandas()

In [ ]:
dv_oxygen_delivery_device = spark.sql("""
    SELECT patient_id AS subject_id, encounter_id AS stay_id, CAST(charttime AS TIMESTAMP) as charttime, value AS o2_delivery_device FROM rv_oxygen_delivery_device
""")
dv_oxygen_delivery_device.createOrReplaceTempView('dv_oxygen_delivery_device')
dv_oxygen_delivery_device.toPandas()

In [ ]:
# Alternative way to get the first intervention
stays_with_interventions = spark.sql("""
WITH odd AS (
    SELECT subject_id, stay_id, charttime, o2_delivery_device AS o2_delivery_device FROM dv_oxygen_delivery_device
), odd1 AS (SELECT *, 
    CASE
    -- tracheostomy
    WHEN o2_delivery_device IN (
        'Tracheostomy tube',
        'Trach mask ' -- 16435 observations
        -- 'T-piece', -- 1135 observations (T-piece could be either InvasiveVent or Tracheostomy)
        ) THEN 'Tracheostomy'
    -- mechanical / invasive ventilation
    WHEN o2_delivery_device IN (
        'Endotracheal tube'
        ) THEN 'InvasiveVent'
    -- NIV
    WHEN o2_delivery_device IN (
        'Bipap mask ', -- 8997 observations
        'CPAP mask ' -- 5568 observations
        ) THEN 'NonInvasiveVent'
    -- high flow nasal cannula
    when o2_delivery_device IN (
        'High flow nasal cannula' -- 925 observations
        ) THEN 'HFNC'
    -- non rebreather
    WHEN o2_delivery_device IN ( 
        'Non-rebreather', -- 5182 observations
        'Face tent', -- 24601 observations
        'Aerosol-cool', -- 24560 observations
        'Venti mask ', -- 1947 observations
        'Medium conc mask ', -- 1888 observations
        'Ultrasonic neb', -- 9 observations
        'Vapomist', -- 3 observations
        'Oxymizer', -- 1301 observations
        'High flow neb', -- 10785 observations
        'Nasal cannula'
        ) THEN 'SupplementalOxygen'
    WHEN o2_delivery_device in (
        'None'
     ) THEN 'None'
    -- not categorized: other
    ELSE NULL END AS ventilation_status
FROM odd WHERE o2_delivery_device  IS NOT NULL
), ventilation_type AS (
SELECT subject_id, stay_id, charttime AS starttime, ventilation_status FROM odd1
), iws AS (
    SELECT stay_id, starttime as  inttime, ventilation_status AS int_type,
row_number() OVER (PARTITION BY stay_id ORDER BY starttime) AS int_sequence
FROM ventilation_type
WHERE ventilation_status NOT in ('None', 'SupplementalOxygen') AND ventilation_status IS NOT NULL
)
SELECT stay_id, inttime, int_type FROM iws WHERE int_sequence = 1
""")
stays_with_interventions.count()

## Observations 
### Chartevents

In [ ]:
def obs_for_coding(system, code, view_name=None):
    obs_view = data.view('Observation',
                         select=[
                             {
                                 "column": [
                                     {
                                         "name": "id",
                                         "path": "id",
                                         "type": "id",
                                         "collection": False
                                     },
                                     {
                                         "name": "patient_id",
                                         "path": "subject.getReferenceKey()",
                                         "type": "string",
                                         "collection": False
                                     },
                                     {
                                         "name": "encounter_id",
                                         "path": "encounter.getReferenceKey()",
                                         "type": "string",
                                         "collection": False
                                     },
                                     {
                                         "name": "date",
                                         "path": "effectiveDateTime",
                                         "type": "dateTime",
                                         "collection": False
                                     },
                                     {
                                         "name": "value",
                                         "path": "valueQuantity.value",
                                         "type": "decimal",
                                         "collection": False
                                     },
                                     {
                                         "name": "unit",
                                         "path": "valueQuantity.unit",
                                         "collection": False
                                     }
                                 ]
                             }
                         ],
                         where=[
                             {
                                 "path": f"code.coding.system = '{system}'"
                             },
                             {
                                 "path": f"code.coding.code = '{code}'"
                             },

                         ]
                         )
    if view_name is not None:
        obs_view.createOrReplaceTempView(view_name)
    return obs_view


CS_MIMIC_CHARTEVENTS_D_ITEMS = 'http://mimic.mit.edu/fhir/mimic/CodeSystem/mimic-chartevents-d-items'
CS_MIMIC_D_LABITEMS = 'http://mimic.mit.edu/fhir/mimic/CodeSystem/mimic-d-labitems'


def obs_chartevent_for_code(code, view_name=None):
    return obs_for_coding(CS_MIMIC_CHARTEVENTS_D_ITEMS, code, view_name)


def obs_labitems_for_code(code, view_name=None):
    return obs_for_coding(CS_MIMIC_D_LABITEMS, code, view_name)

In [ ]:
obs_resp_rate = obs_chartevent_for_code('224690', 'obs_resp_rate')
obs_resp_rate.show()

In [ ]:
obs_spo2 = obs_chartevent_for_code('220277', 'obs_spo2')
obs_spo2.show()

In [ ]:
obs_heart_rate = obs_chartevent_for_code('220045', 'obs_heart_rate')
obs_heart_rate.show()

In [ ]:
#
# alike mimiciv_derrived.vital_signs
#
derr_vital_signs_view = spark.sql("""
WITH ou AS (
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, value AS resp_rate, NULL AS heart_rate, NULL AS spo2 FROM obs_resp_rate 
UNION  
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, NULL AS resp_rate, value AS heart_rate,  NULL AS spo2 FROM obs_heart_rate 
UNION  
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, NULL AS resp_rate, NULL AS heart_rate,  value AS spo2 FROM obs_spo2 
) 
SELECT patient_id, encounter_id, obs_time, FIRST(resp_rate) AS resp_rate, FIRST(heart_rate) AS heart_rate, FIRST(spo2) AS spo2 FROM ou 
GROUP BY patient_id, encounter_id, obs_time
""")
derr_vital_signs_view.sample(0.02).toPandas()

### Labevents

In [ ]:
# bg so2
obs_bg_so2 = obs_labitems_for_code('50817', 'obs_bg_so2')
# AND < 100
obs_bg_so2.show()

In [ ]:
# bg so2
obs_bg_pco2 = obs_labitems_for_code('50818', 'obs_bg_pco2')
obs_bg_pco2.show()

In [ ]:
# hemoglobin
obs_bg_hemoglobin = obs_labitems_for_code('50811', 'obs_bg_hemoglobin')
obs_bg_hemoglobin.toPandas()

In [ ]:
#
# alike mimiciv_derrived.bg
#
derr_bg_view = spark.sql("""
WITH ou AS (
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, value AS so2, NULL AS pco2, NULL AS hemoglobin  FROM obs_bg_so2 
UNION  
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, NULL AS so2, value AS pco2, NULL AS hemoglobin FROM obs_bg_pco2
UNION  
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, NULL AS so2, NULL AS pco2, value AS hemoglobin FROM obs_bg_hemoglobin  
) 
SELECT patient_id, encounter_id, obs_time, FIRST(so2) AS so2, FIRST(pco2) AS pco2, FIRST(hemoglobin) AS hemoglobin FROM ou 
GROUP BY patient_id, encounter_id, obs_time
""")
derr_bg_view.sample(0.02).toPandas()

## Encounters (ICU) 